<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/BERT/BERT%20Training%20more%20than%20512%20sequence%20length.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram

### Connecting to Kaggle

In [1]:
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


### Downloading the dataset

In [2]:
!kaggle datasets download -d datasnaek/mbti-type

 37% 9.00M/24.4M [00:00<00:00, 73.0MB/s]
100% 24.4M/24.4M [00:00<00:00, 120MB/s] 


In [3]:
!unzip /content/mbti-type.zip

Archive:  /content/mbti-type.zip
  inflating: mbti_1.csv              


### Setup

In [ ]:
!pip install transformers
!pip install pyyaml==5.4.1

### Importing Dependencies

In [69]:
import pandas as pd
import numpy as np

import os
from pathlib import Path
from datetime import datetime
import re

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, Input

from transformers import AutoTokenizer, BertTokenizer
from transformers import TFAutoModelForSequenceClassification, TFBertModel

import spacy
from tqdm import tqdm

### Loading Dataset

In [6]:
df = pd.read_csv("/content/mbti_1.csv")

In [7]:
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


### Check if TPU is available

In [8]:
use_tpu = True

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.53.137.170:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.53.137.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.137.170:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


### Pie Chart for Type Column

In [10]:
px.pie(df,names='type',title='Distribution of personality types',hole=0.3)

Data is `extremely imbalanced`. This might cause overfitting to happen since the total data only amounts to 8675.



In [11]:
df['type'].value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

### Cleaning Dataset

In [15]:
nlp = spacy.load("en", parser=False, entity=False)

In [18]:
doc = nlp("a... ok")

In [21]:
for w in doc:
  print(w.lemma_, w.is_stop)

a True
... False
ok False


In [22]:
def clean_text(text):

  #removing links from text data
  text = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',text)

  #removing other symbols
  text = re.sub('[^0-9a-z]',' ',text)

  text = re.sub("\s+", " ", text)
  
  return text

In [23]:
df["cleaned"] = df["posts"].apply(clean_text)

### Splitting Data into Train/Test/Split

In [76]:
le = LabelEncoder()
df["type"] = le.fit_transform(df["type"])

In [77]:
from sklearn.model_selection import train_test_split

posts = df['cleaned'].values
labels =  df['type'].values

train_data, test_data = train_test_split(df, random_state=0, test_size=0.2)

train_size = len(train_data)
test_size = len(test_data)
train_size, test_size

(6940, 1735)

In [32]:
len("oo ".split()) 

1

### Checking MAX LENGTH for BERT Tokenizer

In [33]:
df["length"] = df["cleaned"].apply(lambda x: len(x.split()))

In [34]:
df["length"]

0        573
1       1139
2        855
3       1073
4        991
        ... 
8670     821
8671    1305
8672     971
8673    1656
8674    1309
Name: length, Length: 8675, dtype: int64

In [36]:
px.box(df["length"])

By looking at the graph we can take max sequence length of `1800`

### Initialize Bert tokenizer and masks


In [64]:
bert_model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 1800

def tokenize_sentences(sentences, tokenizer, max_seq_len = 1800):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                            truncation=True,
                            padding="max_length"
                    )
        
        tokenized_sentences.append(tokenized_sentence)
        
    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

In [65]:
train_input_ids = tokenize_sentences(train_data['cleaned'], tokenizer, MAX_LEN)
# train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_attention_masks = create_attention_masks(train_input_ids)

test_input_ids = tokenize_sentences(test_data['cleaned'], tokenizer, MAX_LEN)
# test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_attention_masks = create_attention_masks(test_input_ids)

100%|██████████| 1735/1735 [01:00<00:00, 28.46it/s]


### Define functions for evaluation

In [68]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

### Model Building

In [70]:
def build_model(): 

    input_ids = tf.keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32,
                                           name="input_ids")
    
    bert_layer = TFBertModel.from_pretrained('bert-base-uncased')
    bert_outputs = bert_layer(input_ids)[0]

    pred = tf.keras.layers.Dense(16, activation='softmax')(bert_outputs[:,0,:])
    
    model = tf.keras.models.Model(inputs=input_ids, outputs=pred)

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00002), metrics=['accuracy', f1_m, precision_m, recall_m])

    return model


In [79]:
y_train = train_data["type"].values
y_test = test_data["type"].values

### Model Training

In [80]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = build_model()
else:
    model = build_model()
    
model.summary()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.53.137.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.137.170:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 1800)]            0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             1800, 768),                         
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [ ]:
history = model.fit(np.array(train_input_ids), y_train, verbose = 1, epochs = 10,
                    batch_size = 32,
                    callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5)])

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning:

"`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"



 23/217 [==>...........................] - ETA: 2:34 - loss: 2.4242 - accuracy: 0.1875 - f1_m: 10.7781 - precision_m: 211902208.0000 - recall_m: 5.3890